In [ ]:
import pandas as pd
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Read the train and test paths
df_train = pd.read_csv(train_file_path, sep = '\t', names = ["type", "message"])
df_test = pd.read_csv(test_file_path, sep = '\t', names = ["type", "message"])
df_train.columns = df_train.columns.str.strip()

# Shuffle the train dataset
df_train = df_train.sample(frac = 1)

df_train

In [ ]:
# Split the data into trainx, trainy, testx, testy
df_train_y = df_train.pop('type')
df_train_x = df_train
df_test_y = df_test.pop('type')
df_test_x = df_test

# Convert ham to 0, spam to 1
df_train_y.replace({"ham": 0, "spam": 1}, inplace = True)
df_test_y.replace({"ham": 0, "spam": 1}, inplace = True)
df_train_y

In [ ]:
# Preprocessing function for a pandas dataframe with single column
max_words = 1000

# Create tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train_x["message"])

# For train x and test x only
def preprocess(X):
  # Tokenize the text
  df_t = tokenizer.texts_to_sequences(X["message"])
  X["message"] = df_t

  # Pad the sequences so the array have same length every row
  padded = pad_sequences(X["message"], maxlen=500)
  return padded

In [ ]:
# Tokenize the x data to convert words into numbers
df_train_x = preprocess(df_train_x)
df_test_x = preprocess(df_test_x)
df_test_x

In [ ]:
# Convert dataframe to np array
train_dataset = df_train_x
test_dataset = df_test_x
train_labels = df_train_y
test_labels = df_test_y

train_dataset

In [ ]:
# Build model
max_len = train_dataset.shape[1]

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words,
                              output_dim=64,
                              input_shape = (max_len, )),
    tf.keras.layers.LSTM(64),  # LSTM layer for text processing
    tf.keras.layers.Dropout(0.5),   # Prevent overfitting
    tf.keras.layers.Dense(32, activation = "relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile model
model.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    metrics=["accuracy"]
)


model.summary()

In [ ]:
# Fit model
history = model.fit(
    x = train_dataset,
    y = train_labels,
    epochs = 8,
    validation_split = 0.2
)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Preprocess the text the same way (Tokenize and pad)
  tokens = tokenizer.texts_to_sequences([pred_text])
  padded = pad_sequences(tokens, maxlen = max_len)
  prediction = model.predict(padded)[0][0]

  ans = ([prediction, "spam" if prediction>=0.5 else "ham"])
  print(ans)
  return ans

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
